In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

# 1. Define paths for ImageNet validation images and annotations
IMAGE_DIR = "/home/kajm20/mnist/ILSVRC/Data/CLS-LOC/val"  # Path to validation images
ANNOTATION_DIR = "/home/kajm20/mnist/ILSVRC/Annotations/CLS-LOC/val"  # Path to XML annotations

imagenet_transform = transforms.Compose([
    transforms.Resize(256),  
    transforms.CenterCrop(224),  
    transforms.ToTensor()  # Only convert to tensor (values in range [0,1])
])

# 3. Load the synset mapping
synset_mapping_path = "/home/kajm20/mnist/ILSVRC/LOC_synset_mapping.txt"
wordnet_to_imagenet = {}

# Load synset mapping from file
with open(synset_mapping_path) as f:
    for idx, line in enumerate(f.readlines()):
        wordnet_id, _ = line.split(' ', 1)  # Get WordNet ID from the line (skip class name)
        wordnet_to_imagenet[wordnet_id] = idx  # Map WordNet ID to class index

# 4. Define the custom dataset class
class ImageNetValDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform

        # Get all annotation file names
        self.annotation_files = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.annotation_files)

    def __getitem__(self, idx):
        # Get annotation file path
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        
        # Parse XML to extract class label
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        wordnet_id = root.find("object").find("name").text  # WordNet ID, e.g., 'n01751748'

        # Use the synset mapping to convert WordNet ID to ImageNet class index
        class_idx = wordnet_to_imagenet.get(wordnet_id, -1)  # Default to -1 if not found (shouldn't happen)

        # Get image filename from XML and construct image path
        image_filename = root.find("filename").text + ".JPEG"
        image_path = os.path.join(self.image_dir, image_filename)

        # Load and transform image
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, class_idx

# 5. Initialize the dataset and dataloader
imagenet_val_dataset = ImageNetValDataset(IMAGE_DIR, ANNOTATION_DIR, transform=imagenet_transform)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 6. Define the model (EfficientNet-B0 with pre-trained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_b0(weights='DEFAULT')  # Load pre-trained EfficientNet-B0 model
model.to(device)
model.eval()  # Set the model to evaluation mode




EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [11]:
batch_sum = torch.zeros(3, 224, 224).to(device)  # Ensure it's on the same device
num_images = 0  # Track total images processed

def evaluate_model(model, dataloader):
    global batch_sum, num_images  # Allow modification of global variables

    with torch.no_grad():  
        for images, labels in tqdm(dataloader):
            images = images.to(device)  # Move to same device as batch_sum
            batch_sum += torch.sum(images, dim=0)  # Accumulate sum across batches
            num_images += images.shape[0]  # Update total image count

evaluate_model(model, imagenet_val_loader)

# Compute the average pixel value across all images
print(num_images)
batch_mean = batch_sum / num_images  # Element-wise division
print(batch_mean.shape)
print(batch_mean)  # Should be [3, 224, 224]


100%|██████████| 1563/1563 [00:52<00:00, 29.76it/s]

50000
torch.Size([3, 224, 224])
tensor([[[0.4840, 0.4845, 0.4852,  ..., 0.4854, 0.4854, 0.4853],
         [0.4837, 0.4843, 0.4852,  ..., 0.4858, 0.4857, 0.4857],
         [0.4842, 0.4848, 0.4855,  ..., 0.4859, 0.4856, 0.4856],
         ...,
         [0.4602, 0.4600, 0.4605,  ..., 0.4602, 0.4602, 0.4605],
         [0.4600, 0.4599, 0.4602,  ..., 0.4599, 0.4598, 0.4601],
         [0.4596, 0.4593, 0.4595,  ..., 0.4599, 0.4598, 0.4601]],

        [[0.4811, 0.4815, 0.4820,  ..., 0.4824, 0.4825, 0.4824],
         [0.4807, 0.4811, 0.4818,  ..., 0.4826, 0.4825, 0.4825],
         [0.4807, 0.4812, 0.4818,  ..., 0.4825, 0.4822, 0.4823],
         ...,
         [0.4412, 0.4410, 0.4413,  ..., 0.4408, 0.4410, 0.4417],
         [0.4413, 0.4411, 0.4411,  ..., 0.4407, 0.4408, 0.4413],
         [0.4410, 0.4406, 0.4405,  ..., 0.4410, 0.4410, 0.4413]],

        [[0.4426, 0.4430, 0.4432,  ..., 0.4434, 0.4436, 0.4437],
         [0.4420, 0.4424, 0.4428,  ..., 0.4433, 0.4434, 0.4437],
         [0.4419, 0.4423, 

In [12]:
for name, layer in model.named_modules():
    if any(p.requires_grad for p in layer.parameters()):  # Default recurse=True
        print(name, "->", type(layer))

 -> <class 'torchvision.models.efficientnet.EfficientNet'>
features -> <class 'torch.nn.modules.container.Sequential'>
features.0 -> <class 'torchvision.ops.misc.Conv2dNormActivation'>
features.0.0 -> <class 'torch.nn.modules.conv.Conv2d'>
features.0.1 -> <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
features.1 -> <class 'torch.nn.modules.container.Sequential'>
features.1.0 -> <class 'torchvision.models.efficientnet.MBConv'>
features.1.0.block -> <class 'torch.nn.modules.container.Sequential'>
features.1.0.block.0 -> <class 'torchvision.ops.misc.Conv2dNormActivation'>
features.1.0.block.0.0 -> <class 'torch.nn.modules.conv.Conv2d'>
features.1.0.block.0.1 -> <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
features.1.0.block.1 -> <class 'torchvision.ops.misc.SqueezeExcitation'>
features.1.0.block.1.fc1 -> <class 'torch.nn.modules.conv.Conv2d'>
features.1.0.block.1.fc2 -> <class 'torch.nn.modules.conv.Conv2d'>
features.1.0.block.2 -> <class 'torchvision.ops.misc.Conv2dNormActivation'>

In [ ]:
import os
import torch
import torch.nn as nn
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm
import inspect  # Import inspect to get source code of forward methods

# 1. Define paths for ImageNet validation images and annotations
IMAGE_DIR = "/home/kajm20/mnist/ILSVRC/Data/CLS-LOC/val"  
ANNOTATION_DIR = "/home/kajm20/mnist/ILSVRC/Annotations/CLS-LOC/val"  

imagenet_transform = transforms.Compose([
    transforms.Resize(256),  
    transforms.CenterCrop(224),  
    transforms.ToTensor()  # Only convert to tensor (values in range [0,1])
])

# 3. Load the synset mapping
synset_mapping_path = "/home/kajm20/mnist/ILSVRC/LOC_synset_mapping.txt"
wordnet_to_imagenet = {}

with open(synset_mapping_path) as f:
    for idx, line in enumerate(f.readlines()):
        wordnet_id, _ = line.split(' ', 1)
        wordnet_to_imagenet[wordnet_id] = idx  

# 4. Define the custom dataset class
class ImageNetValDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.annotation_files = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.annotation_files)

    def __getitem__(self, idx):
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        wordnet_id = root.find("object").find("name").text  

        class_idx = wordnet_to_imagenet.get(wordnet_id, -1)  
        image_filename = root.find("filename").text + ".JPEG"
        image_path = os.path.join(self.image_dir, image_filename)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, class_idx

# 5. Initialize the dataset and dataloader
imagenet_val_dataset = ImageNetValDataset(IMAGE_DIR, ANNOTATION_DIR, transform=imagenet_transform)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=1, shuffle=False, num_workers=4)

# 6. Define the model (EfficientNet-B0 with pre-trained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights='DEFAULT')  
model.to(device)
model.eval()  

# 7. Dictionaries to store activations
activation_shapes = {}  # Stores activation shapes
activation_values = {}  # Stores activation values

# 8. Hook function to store activations for **ALL** layers
def hook_fn(module, input, output):
    layer_name = f"{module.__class__.__name__} ({id(module)})" 
    activation_shapes[layer_name] = output.shape  # Store shape
    activation_values[layer_name] = output.detach().cpu()  # Store values (moved to CPU)

# 9. Recursively register hooks for **all** relevant layers
for name, layer in model.named_modules():
    layer.register_forward_hook(hook_fn)

# 10. Print the forward methods of all layers
def print_forward_methods(model):
    print("\n--- Forward Methods of All Layers ---\n")
    for name, layer in model.named_modules():
        if hasattr(layer, "forward"):  # Ensure the layer has a forward method
            try:
                forward_code = inspect.getsource(layer.forward)
                print(f"\n--- {name} ({layer.__class__.__name__}) ---\n")
                print(forward_code)
            except Exception as e:
                print(f"\n--- {name} ({layer.__class__.__name__}) ---\n")
                print(f"Could not retrieve forward method: {e}")

# Call the function to print forward methods
print_forward_methods(model)

# 11. Define the evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            activation_shapes.clear()  
            activation_values.clear()  
            
            print(f"\nRaw Image Tensor Shape (Before Normalization): {images.shape}")  
            
            outputs = model(images)  

            _, predicted = torch.max(outputs, 1)  
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Print activation shapes
            print("\n--- Layer Activation Shapes ---")
            for layer, shape in activation_shapes.items():
                print(f"{layer}: {shape}")

            break  

    accuracy = (correct / total) * 100
    return accuracy

# 12. Evaluate the model on ImageNet validation set
accuracy = evaluate_model(model, imagenet_val_loader)
print(f"EfficientNet-B0 Top-1 Accuracy on ImageNet: {accuracy:.2f}%")



--- Forward Methods of All Layers ---


---  (EfficientNet) ---

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


--- features (Sequential) ---

    def forward(self, input):
        for module in self:
            input = module(input)
        return input


--- features.0 (Conv2dNormActivation) ---

    def forward(self, input):
        for module in self:
            input = module(input)
        return input


--- features.0.0 (Conv2d) ---

    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)


--- features.0.1 (BatchNorm2d) ---

    def forward(self, input: Tensor) -> Tensor:
        self._check_input_dim(input)

        # exponential_average_factor is set to self.momentum
        # (when it is available) only so that it gets updated
        # in ONNX graph when this node is exported to ONNX.
        if self.momentum is None:
            exponential_average_factor = 0.0
        else:
   

  0%|          | 0/50000 [00:00<?, ?it/s]


Raw Image Tensor Shape (Before Normalization): torch.Size([1, 3, 224, 224])

--- Layer Activation Shapes ---
Conv2d (134778334259792): torch.Size([1, 32, 112, 112])
BatchNorm2d (134778334260128): torch.Size([1, 32, 112, 112])
SiLU (134778334260464): torch.Size([1, 32, 112, 112])
Conv2dNormActivation (134778334259456): torch.Size([1, 32, 112, 112])
Conv2d (134778334131088): torch.Size([1, 32, 112, 112])
BatchNorm2d (134778334131408): torch.Size([1, 32, 112, 112])
SiLU (134778334131728): torch.Size([1, 32, 112, 112])
Conv2dNormActivation (134778334130768): torch.Size([1, 32, 112, 112])
AdaptiveAvgPool2d (134778334251056): torch.Size([1, 32, 1, 1])
Conv2d (134782382727696): torch.Size([1, 8, 1, 1])
SiLU (134778334132048): torch.Size([1, 8, 1, 1])
Conv2d (134778334126608): torch.Size([1, 32, 1, 1])
Sigmoid (134778334261136): torch.Size([1, 32, 1, 1])
SqueezeExcitation (134778334249712): torch.Size([1, 32, 112, 112])
Conv2d (134778334132688): torch.Size([1, 16, 112, 112])
BatchNorm2d (1347

In [54]:
import os
import torch
import torch.nn as nn
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

# 1. Define paths for ImageNet validation images and annotations
IMAGE_DIR = "/home/kajm20/mnist/ILSVRC/Data/CLS-LOC/val"  
ANNOTATION_DIR = "/home/kajm20/mnist/ILSVRC/Annotations/CLS-LOC/val"  

imagenet_transform = transforms.Compose([
    transforms.Resize(256),  
    transforms.CenterCrop(224),  
    transforms.ToTensor()  # Only convert to tensor (values in range [0,1])
])

# 3. Load the synset mapping
synset_mapping_path = "/home/kajm20/mnist/ILSVRC/LOC_synset_mapping.txt"
wordnet_to_imagenet = {}

with open(synset_mapping_path) as f:
    for idx, line in enumerate(f.readlines()):
        wordnet_id, _ = line.split(' ', 1)
        wordnet_to_imagenet[wordnet_id] = idx  

# 4. Define the custom dataset class
class ImageNetValDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.annotation_files = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.annotation_files)

    def __getitem__(self, idx):
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        wordnet_id = root.find("object").find("name").text  

        class_idx = wordnet_to_imagenet.get(wordnet_id, -1)  
        image_filename = root.find("filename").text + ".JPEG"
        image_path = os.path.join(self.image_dir, image_filename)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, class_idx

# 5. Initialize the dataset and dataloader
imagenet_val_dataset = ImageNetValDataset(IMAGE_DIR, ANNOTATION_DIR, transform=imagenet_transform)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=2, shuffle=False, num_workers=0)

# 6. Define the model (EfficientNet-B0 with pre-trained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights='DEFAULT')  
model.to(device)
model.eval()  

activation_shapes = {}  # Stores activation shapes
activation_values = {}  # Stores activation values

# 8. Hook function to store activations for **ALL** layers
def hook_fn(module, input, output):
    layer_name = f"{module.__class__.__name__} ({id(module)})" 
    activation_shapes[layer_name] = output.shape  # Store shape
    activation_values[layer_name] = output.detach().cpu()  # Store values (moved to CPU)

# 9. Recursively register hooks for **all** relevant layers, including `AdaptiveAvgPool2d`
for name, layer in model.named_modules():
    layer.register_forward_hook(hook_fn)

# 10. Define the evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            activation_shapes.clear()  
            activation_values.clear()  
            
            print(f"\nRaw Image Tensor Shape (Before Normalization): {images.shape}")  
            
            outputs = model(images)  

            _, predicted = torch.max(outputs, 1)  
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Print activation shapes
            print("\n--- Layer Activation Shapes ---")
            for layer, shape in activation_shapes.items():
                print(f"{layer}: {shape}")

            break  

    accuracy = (correct / total) * 100
    return accuracy

# 11. Evaluate the model on ImageNet validation set
accuracy = evaluate_model(model, imagenet_val_loader)
print(f"EfficientNet-B0 Top-1 Accuracy on ImageNet: {accuracy:.2f}%")

  0%|          | 0/25000 [00:00<?, ?it/s]


Raw Image Tensor Shape (Before Normalization): torch.Size([2, 3, 224, 224])

--- Layer Activation Shapes ---
Conv2d (134778391459024): torch.Size([2, 32, 112, 112])
BatchNorm2d (134778444331168): torch.Size([2, 32, 112, 112])
SiLU (134778391875408): torch.Size([2, 32, 112, 112])
Conv2dNormActivation (134778391876368): torch.Size([2, 32, 112, 112])
Conv2d (134778091292624): torch.Size([2, 32, 112, 112])
BatchNorm2d (134778091462432): torch.Size([2, 32, 112, 112])
SiLU (134778439509216): torch.Size([2, 32, 112, 112])
Conv2dNormActivation (134778391875888): torch.Size([2, 32, 112, 112])
AdaptiveAvgPool2d (134778091641728): torch.Size([2, 32, 1, 1])
Conv2d (134778091291984): torch.Size([2, 8, 1, 1])
SiLU (134778091641968): torch.Size([2, 8, 1, 1])
Conv2d (134778091292944): torch.Size([2, 32, 1, 1])
Sigmoid (134778091387632): torch.Size([2, 32, 1, 1])
SqueezeExcitation (134778091386736): torch.Size([2, 32, 112, 112])
Conv2d (134778091293264): torch.Size([2, 16, 112, 112])
BatchNorm2d (1347

In [65]:
for layer in model.modules:
    print(layer)

TypeError: 'method' object is not iterable

In [71]:
def print_modules(module, name="", indent=0):
    indent_str = "  " * indent
    module_info = str(module).split("\n")[0]  # Get only the first line of module info

    print(f"{indent_str}{name} ({module_info})")  # Print module with full info

    # Recursively print child modules
    for child_name, child in module.named_children():
        full_child_name = f"{name}.{child_name}" if name else child_name
        print_modules(child, full_child_name, indent=indent + 1)

print_modules(model)



 (EfficientNet()
  features (Sequential()
    features.0 (Conv2dNormActivation()
      features.0.0 (Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False))
      features.0.1 (BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
      features.0.2 (SiLU(inplace=True))
    features.1 (Sequential()
      features.1.0 (MBConv()
        features.1.0.block (Sequential()
          features.1.0.block.0 (Conv2dNormActivation()
            features.1.0.block.0.0 (Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False))
            features.1.0.block.0.1 (BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
            features.1.0.block.0.2 (SiLU(inplace=True))
          features.1.0.block.1 (SqueezeExcitation()
            features.1.0.block.1.avgpool (AdaptiveAvgPool2d(output_size=1))
            features.1.0.block.1.fc1 (Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1)))
           

In [59]:
import sys
sys.stdout.flush()

# 7. Dictionaries to store activations
activation_shapes = {}  # Stores activation shapes
activation_values = {}  # Stores activation values

# 8. Hook function to store activations for **ALL** layers
def hook_fn(module, input, output):
    layer_name = f"{module.__class__.__name__} ({id(module)})" 
    activation_shapes[layer_name] = output.shape  # Store shape

    if layer_name not in activation_values:
        activation_values[layer_name] = output.detach().cpu()  # Store values (moved to CPU)
    else:
        activation_values[layer_name] += output.detach().cpu()
    
    if layer_name == 'Conv2d (134778391459024)':
        print(f"Output shape: {output.shape}")
        print(f"Output Values: {output}")

# 9. Recursively register hooks for **all** relevant layers, including `AdaptiveAvgPool2d`
for name, layer in model.named_modules():
    layer.register_forward_hook(hook_fn)

# 10. Define the evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0
    i = 0

    with torch.no_grad():  
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device) 
            
            outputs = model(images)  

            _, predicted = torch.max(outputs, 1)  
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            print('act shape', activation_values['Conv2d (134778391459024)'].shape)
            print('act vals: ', activation_values['Conv2d (134778391459024)'])

            i += 1
            if i == 2:
                break

    accuracy = (correct / total) * 100
    return accuracy

# 11. Evaluate the model on ImageNet validation set
accuracy = evaluate_model(model, imagenet_val_loader)
print(f"EfficientNet-B0 Top-1 Accuracy on ImageNet: {accuracy:.2f}%")


  0%|          | 1/25000 [00:00<1:10:41,  5.89it/s]

Output shape: torch.Size([2, 32, 112, 112])
Output Values: tensor([[[[-7.0785e+00, -6.7842e+00, -6.1812e+00,  ..., -8.7585e+00,
           -8.6696e+00, -8.6657e+00],
          [-1.6641e-01, -2.3652e-01, -2.7026e-01,  ...,  2.2472e-02,
           -2.9725e-02, -4.2478e-02],
          [-6.6271e-01, -7.3188e-01, -7.8982e-01,  ..., -1.5441e-02,
           -3.3033e-03, -2.2467e-02],
          ...,
          [ 6.3514e-02, -1.8203e-01, -1.1913e-01,  ..., -6.3076e-02,
            1.2491e-01, -2.8271e-01],
          [-4.0747e-02, -6.5778e-02, -1.8954e-02,  ..., -3.3597e-01,
            1.3512e-01,  3.7420e-01],
          [-7.8512e-02,  2.4425e-01, -3.9959e-02,  ...,  7.8515e-02,
            1.8995e-01,  3.1853e-01]],

         [[-8.9692e+00,  1.6720e-01, -2.7105e-02,  ..., -2.7308e-01,
           -1.4236e-01, -1.8974e-01],
          [-9.0568e+00,  2.7883e-01,  8.4846e-02,  ...,  3.5232e-02,
            4.3010e-02,  6.9257e-02],
          [-1.0534e+01, -1.7742e-01,  4.4694e-02,  ..., -3.1675e-03,

In [33]:
activation_shapes['Conv2d (134778391459024)']

torch.Size([2, 32, 112, 112])

In [24]:
import functools

activation_shapes = {}  # Stores activation shapes
activation_values = {}  # Stores activation values

# Hook function to store activations for **ALL** layers
def hook_fn(module, input, output, layer_name):
    print(f"Hook triggered for {layer_name}")  # Debug print to ensure hook is called
    activation_shapes[layer_name] = output.shape  # Store shape

    if layer_name not in activation_values:
        activation_values[layer_name] = output.detach().cpu()  # Store values (moved to CPU)
    else:
        activation_values[layer_name] += output.detach().cpu()  # Accumulate if already exists

for name, layer in model.named_modules():
    print(f"Registering hook for layer: {name}")  # Debugging print
    layer.register_forward_hook(functools.partial(hook_fn, layer_name=name))

activation_values

Registering hook for layer: 
Registering hook for layer: features
Registering hook for layer: features.0
Registering hook for layer: features.0.0
Registering hook for layer: features.0.1
Registering hook for layer: features.0.2
Registering hook for layer: features.1
Registering hook for layer: features.1.0
Registering hook for layer: features.1.0.block
Registering hook for layer: features.1.0.block.0
Registering hook for layer: features.1.0.block.0.0
Registering hook for layer: features.1.0.block.0.1
Registering hook for layer: features.1.0.block.0.2
Registering hook for layer: features.1.0.block.1
Registering hook for layer: features.1.0.block.1.avgpool
Registering hook for layer: features.1.0.block.1.fc1
Registering hook for layer: features.1.0.block.1.fc2
Registering hook for layer: features.1.0.block.1.activation
Registering hook for layer: features.1.0.block.1.scale_activation
Registering hook for layer: features.1.0.block.2
Registering hook for layer: features.1.0.block.2.0
Regis

{}

In [23]:
for name, layer in model.named_modules():
    print(name) 


features
features.0
features.0.0
features.0.1
features.0.2
features.1
features.1.0
features.1.0.block
features.1.0.block.0
features.1.0.block.0.0
features.1.0.block.0.1
features.1.0.block.0.2
features.1.0.block.1
features.1.0.block.1.avgpool
features.1.0.block.1.fc1
features.1.0.block.1.fc2
features.1.0.block.1.activation
features.1.0.block.1.scale_activation
features.1.0.block.2
features.1.0.block.2.0
features.1.0.block.2.1
features.1.0.stochastic_depth
features.2
features.2.0
features.2.0.block
features.2.0.block.0
features.2.0.block.0.0
features.2.0.block.0.1
features.2.0.block.0.2
features.2.0.block.1
features.2.0.block.1.0
features.2.0.block.1.1
features.2.0.block.1.2
features.2.0.block.2
features.2.0.block.2.avgpool
features.2.0.block.2.fc1
features.2.0.block.2.fc2
features.2.0.block.2.activation
features.2.0.block.2.scale_activation
features.2.0.block.3
features.2.0.block.3.0
features.2.0.block.3.1
features.2.0.stochastic_depth
features.2.1
features.2.1.block
features.2.1.block

In [15]:
for name, layer in model.named_modules():
    print(name)
    break

In [90]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [94]:
model.features[0][0]

Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [93]:
model.features[0][0].weight.shape

torch.Size([32, 3, 3, 3])

In [97]:
model.features[0][0].weight[0][0]

tensor([[ 0.1216,  0.6563,  0.4567],
        [-0.1109, -0.6100, -0.3334],
        [ 0.0280, -0.1031, -0.1032]], grad_fn=<SelectBackward0>)